# Load Data
This notebook loads and preproceses all necessary data, namely the following.
* OpenWebTextCorpus: for base DistilBERT model
* SQuAD datasrt: for Q&A
* Natural Questions (needs to be downloaded externally but is preprocessed here): for Q&A
* HotPotQA: for Q&A

In [2]:
from tqdm.auto import tqdm
from datasets import load_dataset
import os
import pandas as pd
import random

## Distilbert Data
In the following, we download the english openwebtext dataset from huggingface (https://huggingface.co/datasets/openwebtext). The dataset is provided by Aaron Gokaslan and Vanya Cohen from Brown University (https://skylion007.github.io/OpenWebTextCorpus/).

We first load the data, investigate the structure and write the dataset into files of each 10 000 texts.

In [3]:
ds = load_dataset("openwebtext")

Generating train split: 100%|██████████| 8013769/8013769 [21:29<00:00, 6215.85 examples/s]


In [4]:
# we have a text-only training dataset with 8 million entries
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8013769
    })
})

In [5]:
# create necessary folders
os.mkdir('data')
os.mkdir('data/original')

In [6]:
# save text in chunks of 10000 samples
text = []
ind = 0

for sample in tqdm(ds['train']):
    # replace all newlines
    sample = sample['text'].replace('\n','')
    
    # append cleaned sample to all texts
    text.append(sample)
    
    # if we processed 10000 samples, write them to a file and start over
    if len(text) == 10000:
        with open(f"data/original/text_{ind}.txt", 'w', encoding='utf-8') as f:
            f.write('\n'.join(text))
        text = []
        ind += 1

# write remaining samples to a file
with open(f"data/original/text_{ind}.txt", 'w', encoding='utf-8') as f:
    f.write('\\n'.join(text))

100%|██████████| 8013769/8013769 [05:00<00:00, 26708.01it/s]


### Testing
If we load the first file, we should get a file that is 10000 lines long and has one column

As we do not preprocess the data in any way, but just write the read text into the file, this is all testing necessary

In [9]:
with open("data/original/text_0.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines = pd.DataFrame(lines)

In [10]:
assert lines.shape==(10000,1)
print("Passed")

Passed


## SQuAD Data
In the following, we download the SQuAD dataset from huggingface (https://huggingface.co/datasets/squad). It was initially provided by Rajpurkar et al. from Stanford University.

We again load the dataset and store it in chunks of 1000 into files.

In [11]:
dataset = load_dataset("squad")

In [12]:
os.mkdir("data/training_squad")
os.mkdir("data/test_squad")

In [13]:
# we already have a training and test split. Each sample has an id, title, context, question and answers.
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [14]:
# answers are provided like that - we need to extract answer_end for the model
dataset['train']['answers'][0]

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}

In [15]:
# column contains the split (either train or validation), save_dir is the directory
def save_samples(column, save_dir):
    text = []
    i = 0

    for sample in tqdm(dataset[column]):
        
        # preprocess the context and question by removing the newlines
        context = sample['context'].replace('\n','')
        question = sample['question'].replace('\n','')

        # get the answer as text and start character index
        answer_text = sample['answers']['text'][0]
        answer_start = str(sample['answers']['answer_start'][0])
        
        text.append([context, question, answer_text, answer_start])

        # we choose chunks of 1000
        if len(text) == 1000:
            with open(f"data/{save_dir}/text_{i}.txt", 'w', encoding='utf-8') as f:
                f.write("\n".join(["\t".join(t) for t in text]))
            text = []
            i += 1

    # save remaining
    with open(f"data/{save_dir}/text_{i}.txt", 'w', encoding='utf-8') as f:
        f.write("\n".join(["\t".join(t) for t in text]))

save_samples("train", "training_squad")
save_samples("validation", "test_squad")
    

100%|██████████| 10570/10570 [00:00<00:00, 18865.28it/s]


### Testing
If we load a file, we should get a file with 10000 lines and 4 columns

Also, we want to assure the correct interval. Hence, the second test.

In [16]:
with open("data/training_squad/text_0.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
lines = pd.DataFrame([line.split("\t") for line in lines], columns=["context", "question", "answer", "answer_start"])

In [17]:
assert lines.shape==(1000,4)
print("Passed")

Passed


In [18]:
# we assert that we have the right interval
for ind, line in lines.iterrows():
    sample = line
    answer_start = int(sample['answer_start'])
    assert sample['context'][answer_start:answer_start+len(sample['answer'])] == sample['answer']
print("Passed")

Passed


## Natural Questions Dataset
* Download from https://ai.google.com/research/NaturalQuestions via gsutil (the one from huggingface has 134.92GB, the one from google cloud is in archives)
* Use gunzip to get some samples - we then get `.jsonl`files
* The dataset is a lot more messy, as it is just wikipedia articles with all web artifacts
  * I cleaned the html tags
  * Also I chose a random interval (containing the answer) from the dataset
  * We can't send the whole text into the model anyways

In [19]:
from pathlib import Path
paths = [str(x) for x in Path('data/natural_questions/v1.0/train/').glob('**/*.jsonl')]

In [20]:
os.mkdir("data/natural_questions_train")

In [21]:
import re

# clean html tags
CLEANR = re.compile('<.+?>')
# clean multiple spaces
CLEANMULTSPACE = re.compile('(\s)+')

# the function takes an html documents and removes artifacts
def cleanhtml(raw_html):
    # tags
    cleantext = re.sub(CLEANR, '', raw_html)
    # newlines
    cleantext = cleantext.replace("\n", '')
    # tabs
    cleantext = cleantext.replace("\t", '')
    # character encodings
    cleantext = cleantext.replace("&#39;", "'")
    cleantext = cleantext.replace("&amp;", "'")
    cleantext = cleantext.replace("&quot;", '"')
    # multiple spaces
    cleantext = re.sub(CLEANMULTSPACE, ' ', cleantext)
    # documents end with this tags, if it is present in the string, cut it off
    idx = cleantext.find("<!-- NewPP limit")
    if idx > -1:
        cleantext = cleantext[:idx]
    return cleantext.strip()

In [22]:
import json

# file count
i = 0
data = []

# iterate over all json files
for path in paths:
    print(path)
    # read file and store as list (this requires much memory, as the files are huge)
    with open(path, 'r') as json_file:
        json_list = list(json_file)
    
    # process every context, question, answer pair
    for json_str in json_list:
        result = json.loads(json_str)

        # append a question mark - SQuAD questions end with a qm too
        question = result['question_text'] + "?"
        
        # some question do not contain an answer - we do not need them
        if(len(result['annotations'][0]['short_answers'])==0):
            continue

        # get true start/end byte
        true_start = result['annotations'][0]['short_answers'][0]['start_byte']
        true_end = result['annotations'][0]['short_answers'][0]['end_byte']

        # convert to bytes
        byte_encoding = bytes(result['document_html'], encoding='utf-8')
        
        # the document is the whole wikipedia article, we randomly choose an appropriate part (containing the
        # answer): we have 512 tokens as the input for the model - 4000 bytes lead to a good length
        max_back = 3500 if true_start >= 3500 else true_start
        first = random.randint(int(true_start)-max_back, int(true_start))
        end = first + 3500 + true_end - true_start
        
        # get chosen context
        cleanbytes = byte_encoding[first:end]
        # decode back to text - if our end byte is the middle of a word, we ignore it and cut it off
        cleantext = bytes.decode(cleanbytes, errors='ignore')
        # clean html tags
        cleantext = cleanhtml(cleantext)

        # find the true answer
        answer_start = cleanbytes.find(byte_encoding[true_start:true_end])
        true_answer = bytes.decode(cleanbytes[answer_start:answer_start+(true_end-true_start)])
        
        # clean html tags
        true_answer = cleanhtml(true_answer)
        
        start_ind = cleantext.find(true_answer)
        
        # If cleaning the string makes the answer not findable skip it
        # this hardly ever happens, except if there is an emense amount of web artifacts
        if start_ind == -1:
            continue
            
        data.append([cleantext, question, true_answer, str(start_ind)])

        if len(data) == 1000:
            with open(f"data/natural_questions_train/text_{i}.txt", 'w', encoding='utf-8') as f:
                f.write("\n".join(["\t".join(t) for t in data]))
            i += 1
            data = []
with open(f"data/natural_questions_train/text_{i}.txt", 'w', encoding='utf-8') as f:
    f.write("\n".join(["\t".join(t) for t in data]))

### Testing
In the following, we first check if the shape of the file is correct.

Then we iterate over the file and check if the answers according to the file are the same as in the original file.

In [23]:
with open("data/natural_questions_train/text_0.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
lines = pd.DataFrame([line.split("\t") for line in lines], columns=["context", "question", "answer", "answer_start"])

ValueError: 4 columns passed, passed data had 1 columns

In [28]:
assert lines.shape == (1000, 4)
print("Passed")

Passed


In [34]:
with open("data/natural_questions/v1.0/train/nq-train-00.jsonl", 'r') as json_file:
    json_list = list(json_file)[:500]
del json_file

In [37]:
lines_index = 0
for i in range(len(json_list)):
    result = json.loads(json_list[i])
     
    if(len(result['annotations'][0]['short_answers'])==0):
        pass
    else: 
        # assert that the question text is the same
        assert result['question_text'] + "?" == lines.loc[lines_index, 'question']
        true_start = result['annotations'][0]['short_answers'][0]['start_byte']
        true_end = result['annotations'][0]['short_answers'][0]['end_byte']
        true_answer = bytes.decode(bytes(result['document_html'], encoding='utf-8')[true_start:true_end])
        
        processed_answer = lines.loc[lines_index, 'answer']
        # assert that the answer is the same
        assert cleanhtml(true_answer) == processed_answer
    
        start_ind = int(lines.loc[lines_index, 'answer_start'])
        # assert that the answer (according to the index) is the same
        assert cleanhtml(true_answer) == lines.loc[lines_index, 'context'][start_ind:start_ind+len(processed_answer)]
        
        lines_index += 1
    
    if lines_index == len(lines):
        break
print("Passed")

Passed


## Hotpot QA

In [6]:
ds = load_dataset("hotpot_qa", 'fullwiki')

Found cached dataset hotpot_qa (/home/sabrina/.cache/huggingface/datasets/hotpot_qa/fullwiki/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [19]:
os.mkdir('data/hotpotqa_training')
os.mkdir('data/hotpotqa_test')

In [26]:
# column contains the split (either train or validation), save_dir is the directory
def save_samples(column, save_dir):
    text = []
    i = 0

    for sample in tqdm(ds[column]):
        
        # preprocess the context and question by removing the newlines
        context = sample['context']['sentences']
        context = " ".join(["".join(sentence) for sentence in context])
        question = sample['question'].replace('\n','')
        
        # get the answer as text and start character index
        answer_text = sample['answer']
        answer_start = context.find(answer_text)
        if answer_start == -1:
            continue
            
        
            
        if answer_start > 1500:
            first = random.randint(answer_start-1500, answer_start)
            end = first + 1500 + len(answer_text)
            
            context = context[first:end+1]
            answer_start = context.find(answer_text)
            
            if answer_start == -1:continue
            
        text.append([context, question, answer_text, str(answer_start)])

        # we choose chunks of 1000
        if len(text) == 1000:
            with open(f"data/{save_dir}/text_{i}.txt", 'w', encoding='utf-8') as f:
                f.write("\n".join(["\t".join(t) for t in text]))
            text = []
            i += 1

    # save remaining
    with open(f"data/{save_dir}/text_{i}.txt", 'w', encoding='utf-8') as f:
        f.write("\n".join(["\t".join(t) for t in text]))

save_samples("train", "hotpotqa_training")
save_samples("validation", "hotpotqa_test")

  0%|          | 0/90447 [00:00<?, ?it/s]

  0%|          | 0/7405 [00:00<?, ?it/s]

## Testing

In [27]:
with open("data/hotpotqa_training/text_0.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
lines = pd.DataFrame([line.split("\t") for line in lines], columns=["context", "question", "answer", "answer_start"])

In [28]:
assert lines.shape == (1000, 4)
print("Passed")

Passed


In [29]:
# we assert that we have the right interval
for ind, line in lines.iterrows():
    sample = line
    answer_start = int(sample['answer_start'])
    assert sample['context'][answer_start:answer_start+len(sample['answer'])] == sample['answer']
print("Passed")

Passed
